In [2]:
# import
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [3]:
## load the dataset
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
display(data)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [4]:
## Split the data into training,validation and test sets

train,test=train_test_split(data,test_size=0.25,random_state=42)

In [5]:
train_x=train.drop(['quality'],axis=1).values
train_y=train[['quality']].values.ravel()

## test dataset
test_x=test.drop(['quality'],axis=1).values
test_y=test[['quality']].values.ravel()

## splitting this train data into train and validation

train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature=infer_signature(train_x,train_y)

In [6]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(128,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
                    learning_rate=params["lr"],
                    momentum=params["momentum"]),
                  loss="mean_squared_error",
                  metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [7]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [8]:
space={
        "lr" : hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
        "momentum" : hp.uniform("momentum",0.0,1.0)

}

In [25]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


Epoch 1/3                                            

46/46 [==============================] - 0s 3ms/step - loss: 4.5770 - root_mean_squared_error: 2.1394 - val_loss: 0.5153 - val_root_mean_squared_error: 0.7179

Epoch 2/3                                            

46/46 [==============================] - 0s 1ms/step - loss: 0.5416 - root_mean_squared_error: 0.7359 - val_loss: 0.5069 - val_root_mean_squared_error: 0.7119

Epoch 3/3                                            

46/46 [==============================] - 0s 2ms/step - loss: 0.5187 - root_mean_squared_error: 0.7202 - val_loss: 0.5265 - val_root_mean_squared_error: 0.7256

12/12 [==============================] - 0s 1ms/step - loss: 0.5265 - root_mean_squared_error: 0.7256

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]INFO:tensorflow:Assets written to: C:\Users\mksgh\AppData\Local\Temp\tmpwfe3m4la\model\data\model\assets
Epoch 1/3                                                                      

46/46 [======

In [12]:
## Inferencing

model_uri = 'runs:/171e623bc32c47e898cb22081a66f849/model'

# Replace INPUT_EXAMPLE with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
input_data = test_x

# Verify the model with the provided input data using the logged dependencies.
# For more details, refer to:
# https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
mlflow.models.predict( model_uri=model_uri, input_data=input_data, env_manager="local")

2025/02/01 15:24:20 INFO mlflow.models.python_api: It is highly recommended to use `uv` as the environment manager for predicting with MLflow models as its performance is significantly better than other environment managers. Run `pip install uv` to install uv. See https://docs.astral.sh/uv/getting-started/installation for other installation methods.
2025/02/01 15:24:20 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


39/39 [==============================] - 0s 787us/step
{"predictions": [[6.098855018615723], [7.176075458526611], [6.630058765411377], [5.681721210479736], [6.632232666015625], [5.810438632965088], [5.366516590118408], [5.206330299377441], [5.7631988525390625], [5.53447961807251], [6.412518501281738], [5.090635299682617], [6.8876848220825195], [5.536290645599365], [6.564769268035889], [5.442131996154785], [6.615154266357422], [6.01826810836792], [6.269148826599121], [5.589300632476807], [5.541668891906738], [6.001195907592773], [5.568359851837158], [6.318166732788086], [6.046114444732666], [5.579759120941162], [5.349600315093994], [6.376130104064941], [6.15510368347168], [5.483193874359131], [5.700394630432129], [5.790901184082031], [5.625024318695068], [5.515038967132568], [5.77320671081543], [6.768563270568848], [6.358737945556641], [5.375560283660889], [5.552730083465576], [5.770209789276123], [5.711782932281494], [5.666099548339844], [6.105942726135254], [5.888269424438477], [5.382

In [13]:
# Load model as a PyFuncModel.
model_uri = 'runs:/171e623bc32c47e898cb22081a66f849/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

39/39 [==============================] - 0s 974us/step


array([[6.098855 ],
       [7.1760755],
       [6.630059 ],
       ...,
       [6.4532585],
       [6.8064604],
       [5.964064 ]], dtype=float32)

In [14]:
## Register in the model registry
mlflow.register_model(model_uri,"wine-quality-final")

Successfully registered model 'wine-quality-final'.
Created version '1' of model 'wine-quality-final'.


<ModelVersion: aliases=[], creation_timestamp=1738403728621, current_stage='None', description=None, last_updated_timestamp=1738403728621, name='wine-quality-final', run_id='171e623bc32c47e898cb22081a66f849', run_link=None, source='file:///c:/Users/mksgh/OneDrive%20-%20Vestas%20Wind%20Systems%20A%20S/Documents/Github/mlflow-use-case-ml-dl/DL/mlruns/678676502690910251/171e623bc32c47e898cb22081a66f849/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [27]:
# ## Inferencing

# from mlflow.models import validate_serving_input

# model_uri = 'runs:/171e623bc32c47e898cb22081a66f849/model'

# # The logged model does not contain an input_example.
# # Manually generate a serving payload to verify your model prior to deployment.
# from mlflow.models import convert_input_example_to_serving_input

# # Define INPUT_EXAMPLE via assignment with your own input example to the model
# # A valid input example is a data instance suitable for pyfunc prediction
# serving_payload = convert_input_example_to_serving_input(test_x)

# # Validate the serving payload works on the model
# validate_serving_input(model_uri, serving_payload)


c:\Users\mksgh\OneDrive - Vestas Wind Systems A S\Documents\Github\mlflow-use-case-ml-dl\venv-mlflow\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


39/39 [==============================] - 0s 763us/step


array([[6.098855 ],
       [7.1760755],
       [6.630059 ],
       ...,
       [6.4532585],
       [6.8064604],
       [5.964064 ]], dtype=float32)